In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import conda_installer
from rdkit import Chem
from deepchem.feat.graph_features import atom_features as get_atom_features
import rdkit
import pickle
import matplotlib.pyplot as plt
import deepchem as dc


2023-12-20 19:05:54.021500: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 19:05:54.186762: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 19:05:54.187615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 19:05:55.059270: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
PDBs_folds = pickle.load(open('PDBs_folds_5.pkl', 'rb'))
y_folds = pickle.load(open('y_folds_5.pkl', 'rb'))
x_add_folds = pickle.load(open('x_add_folds_5.pkl', 'rb'))

In [4]:
%cd ../../PGGCN

/home/ali/GBNN/PGGCN


In [6]:
import models.PGNN_model_entropy as pg
import importlib
importlib.reload(pg)

<module 'models.PGNN_model_entropy' from '/home/ali/GBNN/PGGCN/models/PGNN_model_entropy.py'>

In [7]:
hists = []
predicts_train = []
predicts_test = []
loss = []
K = 5
for k in range(K):
    X_train = []
    x_add_train = []
    for i in range(K):
        if i != k:
            X_train += PDBs_folds[i]
            x_add_train += [np.array(x) for x in x_add_folds[i]]
    X_test = PDBs_folds[k]
    x_add_test = x_add_folds[i]
    y_train = np.concatenate(np.array([y_folds[i] for i in range(K) if i != k]), 0)
    y_test = np.array(y_folds[k])
    x_train_parsed, shape_train = pg.data_generator(X_train, x_add_train)
    x_test_parsed, shape_test = pg.data_generator(X_test, x_add_test)
    hs, m = pg.get_trained_model(x_train_parsed, np.array(y_train), shape_train, len(X_train))
    with open('weights_info/PGNN_PG_K%d.pkl' % k, 'wb') as f:
        pickle.dump(m.layers[0].W_list, f)
        f.close()
    predicts_train.append(m.predict(x_train_parsed))
    hists.append(hs)
    loss.append(pg.test_model(x_test_parsed, y_test, len(y_folds[k]), shape_test, m))
    predicts_test.append(x_test_parsed)
    

Epoch 1/25


2023-12-20 19:06:45.761751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-20 19:06:45.833261: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32722200 exceeds 10% of free system memory.


1/1 [==============================] - 5s 5s/step - loss: 13.0841
Epoch 2/25


2023-12-20 19:06:50.977323: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32722200 exceeds 10% of free system memory.
2023-12-20 19:06:50.977427: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32722200 exceeds 10% of free system memory.
2023-12-20 19:06:50.977564: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32722200 exceeds 10% of free system memory.
2023-12-20 19:06:50.977639: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 32722200 exceeds 10% of free system memory.


1/1 [==============================] - 0s 64ms/step - loss: 11.3194
Epoch 3/25
1/1 [==============================] - 0s 60ms/step - loss: 10.0483
Epoch 4/25
1/1 [==============================] - 0s 62ms/step - loss: 8.9686
Epoch 5/25
1/1 [==============================] - 0s 60ms/step - loss: 8.0384
Epoch 6/25
1/1 [==============================] - 0s 65ms/step - loss: 7.2399
Epoch 7/25
1/1 [==============================] - 0s 66ms/step - loss: 6.5820
Epoch 8/25
1/1 [==============================] - 0s 66ms/step - loss: 5.9494
Epoch 9/25
1/1 [==============================] - 0s 67ms/step - loss: 5.4114
Epoch 10/25
1/1 [==============================] - 0s 67ms/step - loss: 4.9597
Epoch 11/25
1/1 [==============================] - 0s 65ms/step - loss: 4.5691
Epoch 12/25
1/1 [==============================] - 0s 63ms/step - loss: 4.2420
Epoch 13/25
1/1 [==============================] - 0s 63ms/step - loss: 3.9937
Epoch 14/25
1/1 [==============================] - 0s 67ms/step - los

In [10]:
m.layers[0].weights[0].shape

TensorShape([75, 64])

In [15]:
with open('PGNN_PG_K5_hists.pkl', 'wb') as f:
    pickle.dump(hists, f)

In [11]:
with open('PGNN_PG_K5_loss.pkl', 'wb') as f:
    pickle.dump(loss, f)

In [12]:
with open('PGNN_PG_K5_predicts_train.pkl', 'wb') as f:
    pickle.dump(predicts_train, f)

In [12]:
predicts_train[0].shape

(80, 1)